In [1]:
import numpy as np
import pandas as pd
import h5py
from scipy.special import xlogy
from sklearn.metrics import accuracy_score

In [2]:
def sigmoid(z):
    return 1/(1+np.power(np.e,-z))

In [3]:
def sigmoid_prime(z):
    return (1-np.power(z,2))

In [4]:
def initialize_neural_network(x):
    np.random.seed(999)
    w=np.random.randn(x.shape[1],1)*0.01
    b=0
    
    return {'w':w,'b':b}

In [20]:
def forward_prop(x,params):
    w=params['w']
    b=params['b']
    z=np.dot(x,w)+b
    a=sigmoid(z)
    
    return {'z':z,'a':a}

In [21]:
def backward_prop(x,y,cache):
    z=cache['z']
    a=cache['a']
    m=x.shape[0]
    dz=a-y
    dw=(1.0/m)*np.dot(x.T,dz)
    db=(1.0/m)*np.sum(dz)
    
    return {'dz':dz,'dw':dw,'db':db}

In [7]:
def update_weights(params,changes,learning_rate=0.001):
    w=params['w']
    b=params['b']
    dw=changes['dw']
    db=changes['db']
    
    w=w-learning_rate*dw
    b=b-learning_rate*db
    
    return {'w':w,'b':b}

In [28]:
def calculate_loss(cache,y):
    a=cache['a']
    m=y.shape[0]
    
    return -1/m*np.sum(xlogy(y,a)+xlogy(1-y,1-a))

In [11]:
# from sklearn.model_selection import train_test_split
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

#     dataSet = pd.read_csv('Data.csv')
#     x = dataSet.iloc[: , :-1].values  # take all line, all colums except last colum from table
#     y = dataSet.iloc[:, -1].values
#     x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)
#     return x_train,y_train,x_test,y_test    

In [32]:
x_train,y_train,x_test,y_test,classes = load_dataset()

In [23]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(209, 64, 64, 3)
(209,)
(50, 64, 64, 3)
(50,)


In [25]:
x_train = x_train.reshape(x_train.shape[0], -1)/255.
x_test = x_test.reshape(x_test.shape[0], -1)/255.
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [26]:
epochs=1000
learning_rate=5e-3

In [30]:
params = initialize_neural_network(x_train)

for i in range(epochs):
    cache=forward_prop(x_train,params)
    loss=calculate_loss(cache,y_train)
    updates=backward_prop(x_train,y_train,cache)
    params=update_weights(params,updates,learning_rate=learning_rate)
    
    if i%(epochs/10)==0:
        print('Epochs: {}\tLoss:{:.5f}'.format(i,loss),end=" ")
        train_cache=np.where(cache['a']>0.5,1,0)
        print('\tTraing accuracy:{:.5f}'.format(accuracy_score(y_train,train_cache)),end=" ")
        test_cache=forward_prop(x_test,params)['a']
        test_cache=np.where(test_cache>=0.5,1,0)
        print('\tTesting accuracy:{:.5f}'.format(accuracy_score(y_test,test_cache)))

Epochs: 0	Loss:0.72463 	Traing accuracy:0.45455 	Testing accuracy:0.34000
Epochs: 100	Loss:0.58224 	Traing accuracy:0.68421 	Testing accuracy:0.34000
Epochs: 200	Loss:0.46335 	Traing accuracy:0.81818 	Testing accuracy:0.44000
Epochs: 300	Loss:0.37246 	Traing accuracy:0.89952 	Testing accuracy:0.62000
Epochs: 400	Loss:0.32947 	Traing accuracy:0.91866 	Testing accuracy:0.70000
Epochs: 500	Loss:0.30155 	Traing accuracy:0.92823 	Testing accuracy:0.72000
Epochs: 600	Loss:0.27835 	Traing accuracy:0.93780 	Testing accuracy:0.74000
Epochs: 700	Loss:0.25866 	Traing accuracy:0.94258 	Testing accuracy:0.74000
Epochs: 800	Loss:0.24167 	Traing accuracy:0.95694 	Testing accuracy:0.74000
Epochs: 900	Loss:0.22683 	Traing accuracy:0.96172 	Testing accuracy:0.76000
